In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from scipy import interpolate
import scipy.integrate
from ww32seismo import *
from os.path import normpath

# Read the observed displacement RMS from the observed data and remove the earthquakes

In [ ]:
import dynamic_yaml
with open("config.yml", 'r') as f:
    configs = dynamic_yaml.load(f)

sta = configs.params.station
net = configs.params.network

seis = pd.read_pickle(normpath("DATA/TEST{}.{}..HHZ.pkl".format(net, sta))) # Read observed data
seis.columns = ["Observed"] # Get same index as model
seis = seis.resample("180T").mean()# Same sampling as P2L model
idx = seis>(seis.std())
seis[idx]=np.NaN

In [ ]:
stations = [sta]
misfits = []
corrs = []

for sta in stations:
    Qs = np.linspace(150,500,36)
    #read DataFrame from pickle file
    for Q in Qs:
        print(int(Q))
        df= pd.read_pickle(normpath("DATA/Q/{}_Q{}.pkl".format(sta, int(Q))))
        
        plot_spec(df, sta, Q)
        
        integ = np.sqrt(scipy.integrate.trapz(df.fillna(0), df.index, axis=0))
        model = pd.DataFrame(integ, index = df.columns)
        model.columns = ["Modeled"]
        pair = seis.merge(model, left_index=True, right_index=True)
        
        pair.plot(title="{}, Q={}".format(sta, int(Q)))
        
        misfit = get_misfit(pair["Observed"], pair["Modeled"])
        misfits.append(misfit)
        corr = get_corr(pair["Observed"], pair["Modeled"])
        corrs.append(corr)

# Look for the best Q for the station

In [ ]:
fig = plt.figure()
plt.scatter(Qs,misfits, c=corrs, cmap="plasma")
plt.colorbar().set_label("Correlation coefficient")
plt.xlabel("Q")
plt.ylabel("misfit")
plt.show()